# Genomic prediction with neural networks
Some preliminary experiments to find theoretical limitations and a suitable network architecture.

In [117]:
import tensorflow as tf
import numpy as np

# build simple network for two marker system (architecture 4,2)
weights = {}
biases = {}
n_markers = 2
archit = [8, 4]
n_layers = len(archit)
n_outputs = 1
initial_learning_rate = 0.01
beta = 0.01

weights['h1'] = tf.Variable(tf.random_normal([n_markers, archit[0]]))
biases['b1'] = tf.Variable(tf.random_normal([archit[0]]))
for l in range(1,n_layers):
    w_name = 'h' + str(l+1)
    b_name = 'b' + str(l+1)
    weights[w_name] = tf.Variable(tf.random_normal([archit[l-1], archit[l]]))
    biases[b_name] = tf.Variable(tf.random_normal([archit[l]]))
weights['out'] = tf.Variable(tf.random_normal([archit[-1], n_outputs]))
biases['out'] = tf.Variable(tf.random_normal([n_outputs]))

nn_x = tf.placeholder(tf.float32, [None, n_markers])
nn_y = tf.placeholder(tf.float32, [None, n_outputs])

layer = nn_x
for l in range(1,n_layers+1):
    weight_name = 'h' + str(l)
    bias_name = 'b' + str(l)
    layer = tf.add(tf.matmul(layer, weights[weight_name]), biases[bias_name])
    ### CHANGE activation function here (sigmoid, relu, elu, tanh, crelu, softsign)
    layer = tf.nn.sigmoid(layer) 
pred = tf.matmul(layer, weights['out']) + biases['out']

loss = tf.reduce_mean(tf.square(pred - nn_y))   # Mean squared error
regularizer = tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2'])
loss = tf.reduce_mean(loss + beta * regularizer)

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 1000, 0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)

# Launch the graph
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [111]:
genos = np.array([[0,0],[0,1],[1,0],[1,1]])
phenos_xor = np.array([[0],[1],[1],[0]])
phenos_xnor = np.array([[1],[0],[0],[1]])
phenos_or = np.array([[0],[1],[1],[1]])
phenos_nor = np.array([[1],[0],[0],[0]])
phenos_and = np.array([[0],[0],[0],[1]])
phenos_add = np.array([[0],[1],[1],[2]])
phenos_xnor_plus_const = np.array([[3],[2],[2],[3]])
phenos = phenos_or

In [114]:
sess.run(optimizer, feed_dict = {nn_x:genos, nn_y:phenos})
c = sess.run(loss,feed_dict = {nn_x:genos, nn_y:phenos})

print("Cost=", "{:.9f}".format(c))

# Predict everything
predicted_phenos = sess.run(pred, feed_dict={nn_x: genos})
print(predicted_phenos)


Cost= 0.557159424
[[1.0852737]
 [1.2009715]
 [1.1823144]
 [1.2715571]]


In [104]:
for i in range(1000):
    sess.run(optimizer, feed_dict = {nn_x:genos, nn_y:phenos})
    c = sess.run(loss,feed_dict = {nn_x:genos, nn_y:phenos})

print("Cost=", "{:.9f}".format(c))

# Predict everything
predicted_phenos = sess.run(pred, feed_dict={nn_x: genos})
print(predicted_phenos)

Cost= 0.031027721
[[0.05178803]
 [0.94714993]
 [0.9471399 ]
 [1.060502  ]]


In [105]:
phenos

array([[0],
       [1],
       [1],
       [1]])

In [106]:
import os,sys,gc
import pandas as pd
import numpy as np
import timeit
from datetime import datetime
import keras
import tensorflow as tf
from keras import backend as K
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise, AlphaDropout, Reshape
from keras.layers import Flatten, LocallyConnected1D, LocallyConnected2D
from keras.optimizers import Adam, Adagrad, Adadelta
from keras.backend.tensorflow_backend import set_session



In [9]:
learning_rate = 0.01
JobID = 1
ps = 25
optim = "adam"
X_file = "KE.geno.csv"
Y_file = "KE_pheno.csv"
CV_file = "KE_cv_pw.csv"
label = "DtSILK"
start_time = timeit.default_timer()
act="sigmoid
drop_rate = str('0.5,0.5,0.5')
arc = str('63,63')
DG = 'D,D,D,D,D,G'
LC = True
training_epochs = 25
hyp = False


In [59]:
def build_network(arc,drop_rate,LC,DG):
    def add_drops(model,drop_out,k):
        if DG[k].upper() == 'D':
            model.add(Dropout(drop_out[0]))
        elif DG[k].upper() == 'G':
            model.add(GaussianNoise(drop_out[k]))
        elif DG[k].upper() == "A":
            model.add(AlphaDropout(drop_out[k]))
        else:
            pass
        return model
    DG = DG.strip().split(",")
    arc = arc.strip().split(",")
    archit = []
    for layer in  arc:
        archit.append(int(layer))
    layer_number = len(archit)
    drop_rate = drop_rate.strip().split(",")
    drop_out = []
    for drops in drop_rate:
        drop_out.append(float(drops))
    model = Sequential()
    model.add(Dense(archit[0], kernel_initializer='truncated_normal', 
                    activation=act, activity_regularizer=keras.regularizers.l1(0.01), input_shape=(x_train.shape[1],)))
    start=1
    for k in range(start,len(archit)):
        model.add(Dense(archit[k], kernel_initializer='truncated_normal', activity_regularizer=keras.regularizers.l1(0.01), activation=act))
        model = add_drops(model,drop_out,k)
    model.add(Dense(1, kernel_initializer='truncated_normal'))
    return(model)




In [60]:
genos = np.array([[0,0],[0,1],[1,0],[1,1]])
phenos_xor = np.array([[0],[1],[1],[0]])
phenos_xnor = np.array([[1],[0],[0],[1]])
phenos_or = np.array([[0],[1],[1],[1]])
phenos_nor = np.array([[1],[0],[0],[0]])
phenos_and = np.array([[0],[0],[0],[1]])
phenos_add = np.array([[0],[1],[1],[2]])
phenos_xnor_plus_const = np.array([[3],[2],[2],[3]])
phenos = phenos_xnor_plus_const
x_train = genos
LC= False

model = build_network(arc,drop_rate,LC,DG)

In [61]:
x_train.shape
arc

'8,4'

In [68]:
arc='8,4'
y_train=phenos_add
model = build_network(arc,drop_rate,LC,DG)
model.compile(loss='mse', optimizer=Adam(lr=0.01),metrics=['accuracy'])
model.fit(x_train,y_train, epochs=50000 , verbose=0)
model.predict(x_train)

array([[0.5005943 ],
       [0.8651502 ],
       [0.92959577],
       [1.5794135 ]], dtype=float32)

In [66]:
phenos_add


array([[0],
       [1],
       [1],
       [2]])

In [43]:
act='sigmoid'
